In diesem Jupyter Notebook beschäftigen wir uns mit der Lemmatisierung einer Dokumentensammlung. Dieser Schritt gehört zum Preprocessing der Dokumente. Die Grundidee der Lemmatisierung ist das Reduzieren von Wörtern auf ihre Stammform, auch Lemma genannt.

Lemmatisierung und Stemming haben gemeinsam, dass sie vor allem Suffixe (Nachsilben) und teilweise auch Präfixe entfernen. Der wesentliche Unterschied zwischen den beiden Verfahren liegt jedoch darin, dass Stemming-Algorithmen ohne Kontext arbeiten. Das bedeutet, dass die resultierenden Wortformen teilweise keinen Sinn ergeben oder falsche Grundformen darstellen. Lemmatisierungsalgorithmen hingegen berücksichtigen den Kontext jedes Wortes im Satz und schlagen es in einem Wörterbuch nach, um die korrekte Grundform zu finden. Dadurch entstehen keine falschen Grundformen.

Beispiel:

Original: "The foxes are running swiftly."
    Stemming: "The fox are run swift."
    Lemmatization: "The fox be run swiftly."

Original: "She enjoys reading books."
    Stemming: "She enjoy read book."
    Lemmatization: "She enjoy reading book."

Wir beschäftigen uns mit den zwei geläufigsten Lemmatizern: dem WordNet Lemmatizer aus der NLTK Library und dem SpaCy Lemmatizer.



In [1]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier nltk spacy


from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

# Loading the NLTK-Ressources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Laden der SpaCy-Ressourcen
!python -m spacy download en_core_web_sm

# Laden des SpaCy-Modells
nlp = spacy.load("en_core_web_sm")

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.2 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


WordNet Lemmatizer

Der WordNet Lemmatizer basiert auf der lexikalischen Datenbank WordNet. Diese besitzt Informationen über die semantischen Relationen zwischen Wörtern. Diese sind in Gruppen bedeutungsähnlicher Wörter eingeteilt und haben z.B. Informationen über Synonyme, Antonyme oder Hyperonyme. Diese Gruppen werden auch Synsets genannt.

Vorgehensweise

1.) Wörterbuchabgleich**:
    Der Lemmatizer prüft ein Wort gegen WordNet, um das Lemma zu finden.

2.) POS-Tag (Part of Speech)**:
    Mit Hilfe des POS-Tags können wir die Wortart eines Wortes im Kontext des Satzes identifizieren und klassifizieren. Wortarten sind z.B. Nomen, Verben, Adverben oder Adjektive. Damit können wir dann die genaue Grundform des Wortes bestimmen.

    Beispiel:
    - Verb: running -> run
    - Nomen: running -> running




        


In [2]:
# Get the describtion of the POS Tags
def get_wordnet_pos_nltk(treebank_tag):
    """Konvertiert POS-Tag in ein Format, das vom WordNet-Lemmatizer unterstützt wird."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [3]:
# Lemmatize a text
def lemmatize_text_nltk(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos_nltk(tag)) for token, tag in pos_tags]
    return ' '.join(lemmatized_tokens)

In [4]:
# Definition der Funktion zur Vorverarbeitung von Dokumenten
def preprocess_documents_nltk(documents):
    """Anwendet Lemmatization auf alle Dokumente."""
    for doc in documents:
        doc['text'] = lemmatize_text_nltk(doc['text'])
        yield doc

In [5]:
# Definition der Funktion zum Erstellen eines Index
def create_index_nltk(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(preprocess_documents_nltk(documents))
    return pt.IndexFactory.of(index_ref)

In [6]:
# Definition der Funktion zum Lesen einer Textdatei und Konvertieren in ein Array
def read_text_file_to_array(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Entfernen von Zeilenumbrüchen und Konvertierung in Strings
            array = [line.strip() for line in lines]
            return array
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

SpaCy Lemmatizer

Der SpaCy Lemmatizer basiert vor allem auf vortrainierten Modellen.

Vorgehensweise

1.) POS (Part of Speech):
        SpaCy identifiziert und klassifiziert Wortarten mit Hilfe von vortrainierten Algorithmen und Modellen.

2.) Lemmatization:
        SpaCy verwendet Regeln und Wortlisten, um das Lemma für ein Wort zu finden. Diese Regeln und Wortlisten sind in den Sprachmodellen von SpaCy eingebettet und basieren auf umfangreichen Trainingsdaten.

3.) Kontext:
        SpaCy berücksichtigt den ganzen Satz und nicht nur das isolierte Wort, um die Lemmas zu finden.



In [7]:
# Definition der Funktion zur Lemmatization eines Textes mit SpaCy
def lemmatize_text_spacy(text):
    """Lemmatiziert den gegebenen Text mit SpaCy."""
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_tokens)

# Definition der Funktion zur Vorverarbeitung von Dokumenten
def preprocess_documents_spacy(documents):
    """Anwendet Lemmatization auf alle Dokumente."""
    for doc in documents:
        doc['text'] = lemmatize_text_spacy(doc['text'])
        yield doc

# Definition der Funktion zum Erstellen eines Index
def create_index_spacy(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(preprocess_documents_spacy(documents))
    return pt.IndexFactory.of(index_ref)


     

In [8]:
# Lesen der Stopwords-Datei und Konvertieren in ein Array
file_path = "../terrier-stopwordlist.txt"
stopwords = read_text_file_to_array(file_path)


costum_index_nltk =create_index_nltk(pt_dataset.get_corpus_iter(), stopwords, 'NoOp')
# Erstellen des benutzerdefinierten Index mit Lemmatization
costum_index_spacy = create_index_spacy(pt_dataset.get_corpus_iter(), stopwords, 'NoOp')

# Erstellen der BatchRetrieve-Instanzen
XSqrA_M = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_nltk = pt.BatchRetrieve(costum_index_nltk, wmodel="XSqrA_M")
XSqrA_M_spacy = pt.BatchRetrieve(costum_index_spacy, wmodel="XSqrA_M")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90009/126958 [07:17<00:25, 1444.97it/s]

09:59:47.699 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [09:10<00:00, 230.48it/s] 


10:01:43.530 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90031/126958 [23:55<02:56, 209.23it/s]

10:25:41.792 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [30:49<00:00, 68.66it/s] 


10:32:37.242 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


In [9]:
# Lemmatisierung der Queries
def lemmatize_queries(queries):
    return [lemmatize_text_spacy(query) for query in queries]

# Erhalten der Topics und Lemmatisierung der Queries
topics_nltk = pt_dataset.get_topics()
topics_nltk['query'] = topics_nltk['query'].apply(lemmatize_text_nltk)
topics_spacy= pt_dataset.get_topics()
topics_spacy['query'] = topics_spacy['query'].apply(lemmatize_text_spacy)

print(topics_nltk['query'])



There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
0     retrieval system improve effectiveness
1      machine learn language identification
2             social medium detect self harm
3                   stem for arabic language
4              audio base animal recognition
                       ...                  
63         information in different language
64                     abbreviation in query
65                   lemmatization algorithm
66                   filter ad rich document
67      advancement in information retrieval
Name: query, Length: 68, dtype: object


In [10]:
print(pt_dataset.get_topics()['query'])

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
0     retrieval system improving effectiveness
1     machine learning language identification
2                social media detect self harm
3                stemming for arabic languages
4               audio based animal recognition
                        ...                   
63           information in different language
64                    abbreviations in queries
65                    lemmatization algorithms
66                    filter ad rich documents
67       advancements in information retrieval
Name: query, Length: 68, dtype: object


In [11]:
# Evaluation der Modelle
pt.Experiment(
    [XSqrA_M, XSqrA_M_nltk, XSqrA_M_spacy],
    topics_nltk,
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names=["XSqrA_M", "XSqrA_M_NLTK", "XSqrA_M_SPACY"]
)



,name,ndcg_cut.10,recip_rank,recall_100,P_10
0,XSqrA_M,0.442880,0.636290,0.587093,0.397059
1,XSqrA_M_NLTK,0.445361,0.653446,0.585188,0.395588
2,XSqrA_M_SPACY,0.442526,0.639003,0.586200,0.395588


In [12]:
# Evaluation der Modelle
pt.Experiment(
    [XSqrA_M, XSqrA_M_nltk, XSqrA_M_spacy],
    topics_spacy,
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names=["XSqrA_M", "XSqrA_M_NLTK", "XSqrA_M_SPACY"]
)

,name,ndcg_cut.10,recip_rank,recall_100,P_10
0,XSqrA_M,0.442880,0.636290,0.587093,0.397059
1,XSqrA_M_NLTK,0.445361,0.653446,0.585188,0.395588
2,XSqrA_M_SPACY,0.442526,0.639003,0.586200,0.395588


In [13]:
# Evaluation der Modelle
pt.Experiment(
    [XSqrA_M, XSqrA_M_nltk, XSqrA_M_spacy],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names=["XSqrA_M", "XSqrA_M_NLTK", "XSqrA_M_SPACY"]
)

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut.10,recip_rank,recall_100,P_10
0,XSqrA_M,0.448211,0.650289,0.592214,0.400000
1,XSqrA_M_NLTK,0.449919,0.667088,0.579003,0.398529
2,XSqrA_M_SPACY,0.444453,0.640535,0.580627,0.398529
